<a href="https://colab.research.google.com/github/rfontanarosa/ground-platform/blob/notebooks/notebooks/generate_offline_imagery1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generate Ground offline imagery (1/2)

This notebook generates a series of sliced cloud-optimized GeoTIFFs (COGs)
for use with the Ground data collection platform Android app.

## Configure all the things

### Required inputs

In [1]:
COUNTRY_NAME=input('Name of country to export, exactly as it appears in FAO GAUL 2015: ')

DEST_BUCKET=input('Bucket ID where GeoTIFFs should be stored: ')

CLOUD_PROJECT=input('Cloud project for Earth Engine execution: ')

Name of country to export, exactly as it appears in FAO GAUL 2015: Côte d'Ivoire
Bucket ID where GeoTIFFs should be stored: ground-raster-basemaps
Cloud project for Earth Engine execution: ground-public-datasets


### Optional config

The constants may be adjusted based on your needs.

In [2]:
START_DATE = '2022-01-01'
END_DATE = '2023-01-01'
DEST_PATH = 'raw-imagery/s2/2022'
OVERWRITE_EXISTING = False

### Fine-tuning

Values have been set through iterative manual experimentation. Only modify these if you know what you're doing!

In [3]:
# A single image is generated for zoom 0..HI_RES_MIN_ZOOM-1.
HI_RES_MIN_ZOOM = 8

# The hi-res images cover zooms HI_RES_MIN_ZOOM..HI_RES_MAX_ZOOM.
HI_RES_MAX_ZOOM = 14

# S2 cloudless export constants:
CLOUD_FILTER = 60
CLD_PRB_THRESH = 40
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 2
BUFFER = 100

# Visualization params:
S2_VIS_PARAMS = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2000, 'gamma': 1.2}
HILLSHADE_VIS_PARAMS = {'min': 0, 'max': 160}
# Ooacity ranges for low and high illumination regions.
LO_MIN = 0
LO_MAX = 170
HI_MIN = 210
HI_MAX = 255
# Value range to stretch opacity [1,0].
LO_OPAC_MIN = -200
LO_OPAC_MAX = 170
HI_OPAC_MIN = 210
HI_OPAC_MAX = 700
LO_VIS = {'min': 0, 'max': 1, 'palette': ['000000']}
HI_VIS = {'min': 0, 'max': 1, 'palette': ['FFFFFF']}

### Debug output

Debug info based on the above constants.

In [4]:
# Compute resolution of images

hi_res_dim = 256 * pow(2, HI_RES_MAX_ZOOM - HI_RES_MIN_ZOOM)
hi_res_pixels = hi_res_dim * hi_res_dim
lo_res_dim = 256 * pow(2, HI_RES_MIN_ZOOM - 1)
lo_res_pixels = lo_res_dim * lo_res_dim

print(f"Hi res images: {hi_res_dim:,} x {hi_res_dim:,} ({hi_res_pixels:,} pixels)")
print(f"Lo res image:  {lo_res_dim:,} x {lo_res_dim:,} ({lo_res_pixels:,} pixels)")

Hi res images: 16,384 x 16,384 (268,435,456 pixels)
Lo res image:  32,768 x 32,768 (1,073,741,824 pixels)


## Initialization and setup

### Install dependencies

In [7]:
!pip install mercantile unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 5.1 MB/s eta 0:00:00


### Do the sign-in boogie

In [6]:
import ee
ee.Authenticate()
ee.Initialize(project=CLOUD_PROJECT)

### Define Folium map utils

Used to preview results on a map.

In [8]:
import folium

def add_ee_feature_collection(self, ee_feature_collection, vis_params, name):
  map_id_dict = ee.FeatureCollection(ee_feature_collection).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True).add_to(self)


# Define a method for displaying Earth Engine image tiles to a folium map.
def add_ee_image(self, ee_image_object, vis_params, name, show=True, opacity=1, min_zoom=0):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        show=show,
        opacity=opacity,
        min_zoom=min_zoom,
        overlay=True,
        control=True
        ).add_to(self)

# Add the Earth Engine layer method to folium.
folium.Map.add_ee_image = add_ee_image

# Add an Earth Engine layer drawing method to folium.
folium.Map.add_ee_feature_collection = add_ee_feature_collection

### Determine extents of tile sets intersecting AOI

In [9]:
import mercantile

def ee_feature_bounds(feature):
  """Synchronously get bounds of EE feature, returning as LngLatBbox.
  """
  coords = feature.bounds().geometry().coordinates().getInfo()[0]
  [west, south], _, [east, north], _, _ = coords
  return mercantile.LngLatBbox(west, south, east, north)

def tile_to_ee_bbox(tile):
  # Note: This can also be done in Earth Engine, but Mercantile provides a
  # more Pythonic API, including model objects like Bounds.
  bounds = mercantile.bounds(tile)
  return ee.Geometry.BBox(bounds.west, bounds.south, bounds.east, bounds.north)

def tile_to_ee_feature(tile):
  return ee.Feature(tile_to_ee_bbox(tile), {'x': tile.x, 'y': tile.y, 'z': tile.z})

def ee_feature_to_tile(ee_feature):
  p = ee_feature['properties']
  return mercantile.Tile(p['x'], p['y'], p['z'])

def filter_tiles_by_bounds(tiles, ee_geometry):
  """Synchronously filter Tiles by EE geometry.
  """
  tiles_fc = ee.FeatureCollection([tile_to_ee_feature(tile) for tile in tiles])
  filtered_fc = tiles_fc.filterBounds(ee_geometry)
  features = filtered_fc.getInfo()['features']
  return [ee_feature_to_tile(f) for f in features]

# Fetch the AOI and bounds.
gaul0 = ee.FeatureCollection("FAO/GAUL/2015/level0")
aoi = ee.Feature(gaul0.filter(ee.Filter.eq('ADM0_NAME', COUNTRY_NAME)).first())
aoi_bounds = ee_feature_bounds(aoi)

# Bounding tiles of all possible tilesets for aoi_bounds.
aoi_bounds_tileset_extents = mercantile.tiles(
    aoi_bounds.west,
    aoi_bounds.south,
    aoi_bounds.east,
    aoi_bounds.north,
    [HI_RES_MIN_ZOOM])

# Bounding tiles only for tilesets that overlap the AOI.
aoi_tileset_extents = filter_tiles_by_bounds(aoi_bounds_tileset_extents, aoi.geometry())

# Display tileset extents on map.
map = folium.Map(width=1024, height=600)

# Show tile extents on map.
for tile in aoi_tileset_extents:
  [west, south, east, north] = mercantile.bounds(tile)
  folium.Rectangle(bounds = [(north, west), (south, east)]).add_to(map)

map.fit_bounds([[aoi_bounds.south, aoi_bounds.west], [aoi_bounds.north, aoi_bounds.east]], padding=(6, 6))

print(f"One GeoTIFF will be exported for each tile at zoom {HI_RES_MAX_ZOOM}, plus one for the entire world at zoom {HI_RES_MAX_ZOOM-1}:")
print()
display(map)

One GeoTIFF will be exported for each tile at zoom 14, plus one for the entire world at zoom 13:



### Define image source

Build cloudless mosaic using publicly available 10m Sentinel-2 imagery.

In [10]:
# Based on https://developers.google.com/earth-engine/tutorials/community/sentinel-2-s2cloudless
def get_s2_sr_cld_col(aoi, start_date, end_date):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)

def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)

####### Hillshade w/ variable transparency #######
# Based on https://gis.stackexchange.com/questions/445241/hillshade-image-blend-that-is-not-washed-out-earth-engine
def apply_hillshade(img):
  exaggeration = 0.75
  hillshade = ee.Terrain.hillshade(
    input=ee.Image('NASA/NASADEM_HGT/001').select('elevation').multiply(exaggeration),
    azimuth=150,
    elevation=30
  ).updateMask(img.select(0).mask())

  # Create low and high illumination hillshade images with transparency
  # gradient controlled by illumination; combine them and blend on top
  # of the original image.
  lo_clamped = hillshade.clamp(LO_MIN, LO_MAX)
  lo_scaled = lo_clamped.unitScale(LO_OPAC_MIN, LO_OPAC_MAX)
  lo = ee.Image(1).subtract(lo_scaled).selfMask().visualize(**LO_VIS)

  hi_clamped = hillshade.clamp(HI_MIN, HI_MAX)
  hi_scaled = hi_clamped.unitScale(HI_OPAC_MIN , HI_OPAC_MAX)
  hi = hi_scaled.selfMask().visualize(**HI_VIS)

  return img.blend(lo.blend(hi))

# Build S2 mosaic.
s2_sr_cld_col = get_s2_sr_cld_col(aoi.geometry(), START_DATE, END_DATE)
s2_sr_median = s2_sr_cld_col.map(add_cld_shdw_mask).map(apply_cld_shdw_mask).median()
image = apply_hillshade(s2_sr_median.visualize(**S2_VIS_PARAMS))

# Preview imagery.
map = folium.Map(width=1024, height=600, start_zoom=9)
map.add_ee_image(image.clip(aoi), {}, 'Image export preview', show=True, opacity=1, min_zoom=9)
map.fit_bounds([[aoi_bounds.south, aoi_bounds.west], [aoi_bounds.north, aoi_bounds.east]])

print("Visualizing preview - this may take awhile. You may proceed with other steps without waiting.")
print()
display(map)

Visualizing preview - this may take awhile. You may proceed with other steps without waiting.



## Export GeoTIFFs

### Prepare task params

In [ ]:
from unidecode import unidecode

def build_export_task(image, extent, max_zoom, file_name_prefix, dimensions):
  x, y, z = extent
  country = unidecode(COUNTRY_NAME).replace("'", "").replace(" ", "-")
  return {
    'fileFormat': 'GeoTIFF',
    'image': image,
     # Note: Region is specified in the source image's CRS, not the target one.
    'region': tile_to_ee_bbox(extent),
    'description': f"Export-{country}-{z}-{x}-{y}",
    'bucket': DEST_BUCKET,
    'fileNamePrefix': file_name_prefix,
    'maxWorkers': 2000,
    'maxPixels': dimensions * dimensions,
    'dimensions': dimensions,
    'fileDimensions': dimensions,
    'crs': "EPSG:3857"
  }

# Hi-res images.
tasks = [build_export_task(
    image=image,
    extent=extent,
    max_zoom=HI_RES_MAX_ZOOM,
    file_name_prefix=f"{DEST_PATH}/{extent.z}/{extent.x}/{extent.y}",
    dimensions=hi_res_dim)
 for extent in aoi_tileset_extents]

print(len(tasks), "task(s) ready to run")

27 task(s) ready to run


### Run tasks

In [ ]:
from google.cloud import storage

# Uncomment to overwrite.
# OVERWRITE_EXISTING=True

# Access dest bucket to check if files already exist.
storage_client = storage.Client(credentials=ee.data._credentials)
bucket = storage_client.bucket(DEST_BUCKET)

# https://google-auth-oauthlib.readthedocs.io/en/latest/reference/google_auth_oauthlib.flow.html
# Export missing tilesets.
print("Starting tasks. See running tasks at https://code.earthengine.google.com/tasks")
for i, t in enumerate(tasks):
  name = t['fileNamePrefix'] + '.tif'
  blob = storage.Blob(name, bucket)
  if (not OVERWRITE_EXISTING and blob.exists(storage_client)):
    print(blob.public_url, 'exists, skipping.')
  else:
    task = ee.batch.Export.image.toCloudStorage(**t)
    task.start()
    print(i, task, name)

print("Export tasks started")


Starting tasks. See running tasks at https://code.earthengine.google.com/tasks
0 <Task RFO7UJCTYPARFDU5Z73DEVMQ Type.EXPORT_IMAGE: Export-Ivory Coast-8-121-122 (State.UNSUBMITTED)> raw-imagery/s2/2022/8/121/122.tif
1 <Task ACSD343DV7HPXG2X4QYQYUQS Type.EXPORT_IMAGE: Export-Ivory Coast-8-121-123 (State.UNSUBMITTED)> raw-imagery/s2/2022/8/121/123.tif
2 <Task HJY6ARNX3AMI3QJ44PGQNVN7 Type.EXPORT_IMAGE: Export-Ivory Coast-8-122-120 (State.UNSUBMITTED)> raw-imagery/s2/2022/8/122/120.tif
3 <Task ZAUASW54OTZ5T6HKVXYFZDTW Type.EXPORT_IMAGE: Export-Ivory Coast-8-122-121 (State.UNSUBMITTED)> raw-imagery/s2/2022/8/122/121.tif
4 <Task ZWFUQH7NLSDEY7VRWY34OS2H Type.EXPORT_IMAGE: Export-Ivory Coast-8-122-122 (State.UNSUBMITTED)> raw-imagery/s2/2022/8/122/122.tif
5 <Task FGCYP4MHOV2JUYVYEXJOQ5PB Type.EXPORT_IMAGE: Export-Ivory Coast-8-122-123 (State.UNSUBMITTED)> raw-imagery/s2/2022/8/122/123.tif
6 <Task AMLJKZRXCMRTB5GRIA7OXY7N Type.EXPORT_IMAGE: Export-Ivory Coast-8-122-124 (State.UNSUBMITTED)> raw

## Next steps

If tasks fail, ensure `OVERWRITE_EXISTING=False` and rerun to generate missing outputs.

Once the above tasks complete, proceed to [Post-process Ground imagery](https://colab.research.google.com/github/google/ground-platform/blob/master/notebooks/generate_offline_imagery2.ipynb) to prepare the generated imagery for use in Ground.